# Topic 5: Bayesian Inference in Regression Models

## Learning Objectives
- Apply Bayesian methods to linear and generalized linear models
- Understand Bayesian logistic and Poisson regression
- Implement model checking and validation
- Handle uncertainty in regression parameters

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pymc as pm
import arviz as az
from sklearn.datasets import make_classification, make_regression

plt.style.use('seaborn-v0_8')
np.random.seed(42)

## 1. Bayesian Linear Regression

### Model:
$$y_i = \alpha + \beta x_i + \epsilon_i, \quad \epsilon_i \sim N(0, \sigma^2)$$

### Priors:
- $\alpha \sim N(0, 10^2)$
- $\beta \sim N(0, 10^2)$  
- $\sigma \sim \text{HalfNormal}(1)$

In [ ]:
# Generate synthetic data
n = 100
x = np.random.uniform(0, 10, n)
true_alpha, true_beta, true_sigma = 2.0, 1.5, 0.8
y = true_alpha + true_beta * x + np.random.normal(0, true_sigma, n)

# Bayesian Linear Regression
with pm.Model() as linear_model:
    # Priors
    alpha = pm.Normal('alpha', 0, 10)
    beta = pm.Normal('beta', 0, 10)
    sigma = pm.HalfNormal('sigma', 1)
    
    # Linear model
    mu = alpha + beta * x
    
    # Likelihood
    y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=y)
    
    # Sample
    trace = pm.sample(2000, return_inferencedata=True, random_seed=42)

# Results
print("Bayesian Linear Regression Results:")
print(f"True parameters: α={true_alpha}, β={true_beta}, σ={true_sigma}")
print("\nPosterior Summary:")
print(az.summary(trace, var_names=['alpha', 'beta', 'sigma']))

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Data and posterior predictions
axes[0,0].scatter(x, y, alpha=0.6, label='Data')

# Posterior predictive samples
alpha_samples = trace.posterior['alpha'].values.flatten()
beta_samples = trace.posterior['beta'].values.flatten()

x_pred = np.linspace(0, 10, 100)
for i in range(0, len(alpha_samples), 100):
    y_pred = alpha_samples[i] + beta_samples[i] * x_pred
    axes[0,0].plot(x_pred, y_pred, 'r-', alpha=0.1)

# True line
y_true = true_alpha + true_beta * x_pred
axes[0,0].plot(x_pred, y_true, 'g--', linewidth=2, label='True relationship')

axes[0,0].set_xlabel('x')
axes[0,0].set_ylabel('y')
axes[0,0].set_title('Bayesian Linear Regression')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Posterior distributions
az.plot_posterior(trace, var_names=['alpha', 'beta'], ax=axes[0,1])
axes[0,1].set_title('Posterior Distributions')

# Trace plots
az.plot_trace(trace, var_names=['alpha', 'beta'], axes=axes[1,:])

plt.tight_layout()
plt.show()

## 2. Bayesian Logistic Regression

### Model:
$$\text{logit}(p_i) = \alpha + \beta x_i$$
$$y_i \sim \text{Bernoulli}(p_i)$$

In [ ]:
# Generate binary classification data
from sklearn.datasets import make_classification
X, y_binary = make_classification(n_samples=200, n_features=1, n_redundant=0, 
                                 n_informative=1, n_clusters_per_class=1, random_state=42)
x_binary = X.flatten()

# Bayesian Logistic Regression
with pm.Model() as logistic_model:
    # Priors
    alpha = pm.Normal('alpha', 0, 2.5)
    beta = pm.Normal('beta', 0, 2.5)
    
    # Logistic model
    logit_p = alpha + beta * x_binary
    p = pm.Deterministic('p', pm.math.sigmoid(logit_p))
    
    # Likelihood
    y_obs = pm.Bernoulli('y_obs', p=p, observed=y_binary)
    
    # Sample
    trace_logistic = pm.sample(2000, return_inferencedata=True, random_seed=42)

print("Bayesian Logistic Regression Results:")
print(az.summary(trace_logistic, var_names=['alpha', 'beta']))

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Data and predictions
axes[0].scatter(x_binary[y_binary==0], y_binary[y_binary==0], alpha=0.6, label='Class 0')
axes[0].scatter(x_binary[y_binary==1], y_binary[y_binary==1], alpha=0.6, label='Class 1')

# Posterior predictive
x_pred = np.linspace(x_binary.min(), x_binary.max(), 100)
alpha_samples = trace_logistic.posterior['alpha'].values.flatten()
beta_samples = trace_logistic.posterior['beta'].values.flatten()

for i in range(0, len(alpha_samples), 200):
    logit_pred = alpha_samples[i] + beta_samples[i] * x_pred
    p_pred = 1 / (1 + np.exp(-logit_pred))
    axes[0].plot(x_pred, p_pred, 'r-', alpha=0.1)

axes[0].set_xlabel('x')
axes[0].set_ylabel('P(y=1)')
axes[0].set_title('Bayesian Logistic Regression')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Posterior distributions
az.plot_posterior(trace_logistic, var_names=['alpha', 'beta'], ax=axes[1])
axes[1].set_title('Posterior Distributions')

# ROC-like analysis
p_samples = trace_logistic.posterior['p'].values.reshape(-1, len(x_binary))
p_mean = p_samples.mean(axis=0)

from sklearn.metrics import roc_curve, auc
fpr, tpr, _ = roc_curve(y_binary, p_mean)
roc_auc = auc(fpr, tpr)

axes[2].plot(fpr, tpr, 'b-', linewidth=2, label=f'ROC (AUC = {roc_auc:.3f})')
axes[2].plot([0, 1], [0, 1], 'k--', alpha=0.5)
axes[2].set_xlabel('False Positive Rate')
axes[2].set_ylabel('True Positive Rate')
axes[2].set_title('ROC Curve')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Bayesian Poisson Regression

### Model:
$$\log(\lambda_i) = \alpha + \beta x_i$$
$$y_i \sim \text{Poisson}(\lambda_i)$$

In [ ]:
# Generate count data
n = 100
x_count = np.random.uniform(0, 5, n)
true_alpha_count, true_beta_count = 0.5, 0.8
lambda_true = np.exp(true_alpha_count + true_beta_count * x_count)
y_count = np.random.poisson(lambda_true)

# Bayesian Poisson Regression
with pm.Model() as poisson_model:
    # Priors
    alpha = pm.Normal('alpha', 0, 2)
    beta = pm.Normal('beta', 0, 2)
    
    # Poisson model
    log_lambda = alpha + beta * x_count
    lambda_param = pm.Deterministic('lambda', pm.math.exp(log_lambda))
    
    # Likelihood
    y_obs = pm.Poisson('y_obs', mu=lambda_param, observed=y_count)
    
    # Sample
    trace_poisson = pm.sample(2000, return_inferencedata=True, random_seed=42)

print("Bayesian Poisson Regression Results:")
print(f"True parameters: α={true_alpha_count}, β={true_beta_count}")
print("\nPosterior Summary:")
print(az.summary(trace_poisson, var_names=['alpha', 'beta']))

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Data and predictions
axes[0].scatter(x_count, y_count, alpha=0.6, label='Data')

# Posterior predictive
x_pred = np.linspace(0, 5, 100)
alpha_samples = trace_poisson.posterior['alpha'].values.flatten()
beta_samples = trace_poisson.posterior['beta'].values.flatten()

for i in range(0, len(alpha_samples), 200):
    lambda_pred = np.exp(alpha_samples[i] + beta_samples[i] * x_pred)
    axes[0].plot(x_pred, lambda_pred, 'r-', alpha=0.1)

# True relationship
lambda_true_pred = np.exp(true_alpha_count + true_beta_count * x_pred)
axes[0].plot(x_pred, lambda_true_pred, 'g--', linewidth=2, label='True λ(x)')

axes[0].set_xlabel('x')
axes[0].set_ylabel('Count')
axes[0].set_title('Bayesian Poisson Regression')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Posterior distributions
az.plot_posterior(trace_poisson, var_names=['alpha', 'beta'], ax=axes[1])
axes[1].set_title('Posterior Distributions')

# Residual analysis
lambda_samples = trace_poisson.posterior['lambda'].values.reshape(-1, len(x_count))
lambda_mean = lambda_samples.mean(axis=0)
residuals = y_count - lambda_mean

axes[2].scatter(lambda_mean, residuals, alpha=0.6)
axes[2].axhline(0, color='red', linestyle='--')
axes[2].set_xlabel('Fitted Values')
axes[2].set_ylabel('Residuals')
axes[2].set_title('Residual Plot')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Key Takeaways

### Advantages of Bayesian Regression:
- **Full uncertainty quantification** for all parameters
- **Natural regularization** through priors
- **Flexible model specification** and extensions
- **Principled model comparison** via information criteria

### Model Checking:
- **Posterior predictive checks** assess model adequacy
- **Residual analysis** identifies model violations
- **Cross-validation** evaluates predictive performance

## Next: Topic 6 - Hierarchical Models